In [5]:
%matplotlib inline

import scipy.io
import matplotlib.pyplot as plt
import scipy
import numpy as np
import os
import itertools

In [42]:
def flatMap(f, items):
        return list(itertools.chain.from_iterable(itertools.imap(f, items)))

In [98]:
def flatten(l):
    return flatMap(lambda x:x, l)

In [7]:
import numpy

def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError, "smooth only accepts 1 dimension arrays."

    if x.size < window_len:
        raise ValueError, "Input vector needs to be bigger than window size."


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"


    s=numpy.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=numpy.ones(window_len,'d')
    else:
        w=eval('numpy.'+window+'(window_len)')

    y=numpy.convolve(w/w.sum(),s,mode='valid')
    return y[(window_len/2-1):-(window_len/2)]


In [60]:
from scipy.signal import argrelmax, argrelmin, find_peaks_cwt

def findLocalExtremeIndices(sig, minGap = None, varPercentage = None, minValue = None, maxValue = None):
    peaks = find_peaks_cwt(sig, np.arange(1,len(sig)*(1.0/100)))
    maximums = argrelmax(sig)[0]
    maximums = map(lambda peak: min(maximums, key=lambda x:abs(x-peak)), peaks)
    minimums = argrelmin(sig)[0]
    minimums = map(lambda peak: min(minimums, key=lambda x:abs(x-peak)), peaks)
    
    minimums = list(set(minimums))
    maximums = list(set(maximums))
    absMin = min([sig[i] for i in minimums])
    absMax = min([sig[i] for i in maximums])
    
    if varPercentage:
        minimums = filter(lambda i: absMin / sig[i] < varPercentage, minimums)
        maximums = filter(lambda i: absMax / sig[i] < varPercentage, maximums)
    if minValue:
        maximums = filter(lambda i: sig[i] > minValue, maximums)
    if maxValue:
        minimums = filter(lambda i: sig[i] < maxValue, minimums)
    
    
    minimums.sort()
    maximums.sort()
    
    if minGap:
        for l in [minimums, maximums]:
            temp = list(set(l)) # to copy the list and not save a ref
            for i in range(1, len(l)):
                if (l[i] - l[i-1]) < minGap:
                    if(sig[l[i]] > sig[l[i-1]]):
                        temp.remove(l[i-1])
                    else:
                        temp.remove(l[i])
            l = temp            
    
    return minimums, maximums

In [9]:
def get_cell_type(cell):
    num = cell['CellNum'][0][0][0][0]
    if num in [19,20,21,22,23,24,36,37,38]:
        return 'SpVi'
    else:
        return 'PrV'

In [10]:
def compute_xvalue_averages_for_one_cell(cell):
    ''' vMat
    A matrix that represesnts the potential responses of the cell.
    80 trials, each 40,000 samples long
    vMat[i][j] := potential in sample i at trial j (mV)

    40,000 samples \ 20KHz sampling rate -> trials are 2 seconds long.

    thus each sample is 2 \ 40000 => 1  \ 20000 => 0.05 ms long
    '''
    vMat = cell['vMat'][0][0].transpose()
    
    ''' stimMat
    A matrix that the galvo feedback (the stimilus to the electical device 
    that mechanically stimulated the whiskers.)
    80 trials, each 8,000 samples long
    stimMat[i][j] := ? amount of stimulus

    8,000 samples \ 4KHz sampling rate -> trials are 2 seconds long.
    thus each sample is 2 \ 8000 = 1 \ 4000 = 0.25ms long
    
     Note that vMat is 5x in the X axis.

    '''
    stimMat = cell['stimMat'][0][0].transpose()
    xValuesVec = cell['xValue'][0][0].tolist()
    
    '''
    xValue is a unique identifier for a stimilus pattern.
    There are 20 (as you can see below) xValues.
    Each trial was done in one of 20 stimulus patterns, and we can check which by using the xValueVec.
    '''
    xValues = np.unique(xValuesVec)
    
    xValueToTrials = dict(map(lambda xValue: (xValue, [i for i in range(0,len(xValuesVec)) if xValuesVec[i] == xValue]), xValues))
    xValueToAverageVoltagesVector = dict(map(lambda xValue: (xValue, np.mean(vMat[xValueToTrials[xValue]], axis = 0)), xValueToTrials))
    xValueToAverageStimulusVector = dict(map(lambda xValue: (xValue, np.mean(stimMat[xValueToTrials[xValue]], axis = 0)), xValueToTrials))

    return (xValueToAverageStimulusVector,xValueToAverageVoltagesVector)

In [11]:
'''
    Overall average computation
'''
filenames = [os.path.join('data', fn) for fn in os.walk('data').next()[2] if fn.endswith('.mat')]
mats = [scipy.io.loadmat(fn) for fn in filenames]
cells = [mat['Cell'] for mat in mats]
xValueAverageVectors = [compute_xvalue_averages_for_one_cell(cell) for cell in cells]

In [12]:
# '''
#     EXAMPLES
#     the following 4 lines are an example for one xvalue of one cell
# '''
# exampleXValue = 1
# (xValueToAverageStimulusVector, xValueToAverageVoltagesVector) =  xValueAverageVectors[0]
# stimsample = xValueToAverageStimulusVector[exampleXValue]
# plt.plot(stimsample)
# plt.show()

In [13]:
# stimsamplesmooth = smooth(stimsample, window_len = 100,window='hanning')
# plt.plot(stimsamplesmooth)
# plt.show()

In [14]:
# vsample = xValueToAverageVoltagesVector[exampleXValue]
# plt.plot(vsample)
# plt.show()

In [15]:
# vsamplesmooth = smooth(vsample, window_len = 500,window='hanning')
# plt.plot(vsamplesmooth)
# plt.show()

In [16]:
spviAverages = [avgs for cell, avgs in itertools.izip(cells, xValueAverageVectors) if get_cell_type(cell) == 'SpVi']
prvAverages = [avgs for cell, avgs in itertools.izip(cells, xValueAverageVectors) if get_cell_type(cell) == 'PrV']

In [17]:
spviXValueToStimulusAverages = map(lambda xvalue: [stimAvgVec[xvalue] for stimAvgVec, voltAvgVec in spviAverages], range(1,21))
spviXValueToVoltageAverages = map(lambda xvalue: [voltAvgVec[xvalue] for stimAvgVec, voltAvgVec in spviAverages], range(1,21))

prvXValueToStimulusAverages = map(lambda xvalue: [stimAvgVec[xvalue] for stimAvgVec, voltAvgVec in prvAverages], range(1,21))
prvXValueToVoltageAverages = map(lambda xvalue: [voltAvgVec[xvalue] for stimAvgVec, voltAvgVec in prvAverages], range(1,21))

In [18]:
spviXValueToStimulusOverallAverageVectors = [smooth(np.mean(v, axis = 0), window_len = 100,window='hanning') for v in spviXValueToStimulusAverages]
spviXValueToVoltageOverallAverageVectors = [smooth(np.mean(v, axis = 0), window_len = 100,window='hanning') for v in spviXValueToVoltageAverages]

prvXValueToStimulusOverallAverageVectors = [smooth(np.mean(v, axis = 0), window_len = 100,window='hanning') for v in prvXValueToStimulusAverages]
prvXValueToVoltageOverallAverageVectors = [smooth(np.mean(v, axis = 0), window_len = 100,window='hanning') for v in prvXValueToVoltageAverages]

In [19]:
# for xvalue in range(0,20):
#     plt.close(1)
#     plt.rc('xtick', labelsize=2.8)
#     plt.plot(spviXValueToStimulusOverallAverageVectors[xvalue])
#     plt.xticks(np.arange(0, len(spviXValueToStimulusOverallAverageVectors[xvalue])+1, 200.0))
#     plt.savefig(r"C:\Users\scaly\Documents\data\graphs\spvi_stim_%d.pdf" % xvalue)
#     plt.close(1)
   
#     plt.rc('xtick', labelsize=2)
#     plt.plot(spviXValueToVoltageOverallAverageVectors[xvalue])
#     plt.xticks(np.arange(0, len(spviXValueToVoltageOverallAverageVectors[xvalue])+1, 1000.0))
#     plt.savefig(r"C:\Users\scaly\Documents\data\graphs\spvi_volt_%d.pdf" % xvalue)
#     plt.close(1)
   
#     plt.rc('xtick', labelsize=2.8)
#     plt.plot(prvXValueToStimulusOverallAverageVectors[xvalue])
#     plt.xticks(np.arange(0, len(prvXValueToStimulusOverallAverageVectors[xvalue])+1, 200.0))
#     plt.savefig(r"C:\Users\scaly\Documents\data\graphs\prv_stim_%d.pdf" % xvalue)
   
#     plt.close(1)
#     plt.rc('xtick', labelsize=2)
#     plt.plot(prvXValueToVoltageOverallAverageVectors[xvalue])
#     plt.xticks(np.arange(0, len(prvXValueToVoltageOverallAverageVectors[xvalue])+1, 1000.0))
#     plt.savefig(r"C:\Users\scaly\Documents\data\graphs\prv_volt_%d.pdf" % xvalue)
#     plt.close(1)

In [20]:
# xvalue = 1
# plt.close(1)
# plt.rc('xtick', labelsize=1.3) 
# plt.plot(spviXValueToStimulusOverallAverageVectors[xvalue])
# plt.xticks(np.arange(0, len(spviXValueToStimulusOverallAverageVectors[xvalue])+1, 100.0))
# plt.savefig(r"C:\Users\scaly\Documents\data\graphs\spvi_stim_%d.pdf" % xvalue, format = "pdf")

In [68]:
spviXValueToStimulusPeaks = [[(0.25*i,v[i]) for i in findLocalExtremeIndices(v, minGap = (25 / 0.25), maxValue = -0.35)[0]] for v in spviXValueToStimulusOverallAverageVectors]

In [79]:
spviXValueToVoltagePeaks = [[(0.05*i,v[i]) for i in findLocalExtremeIndices(v, minValue = -69.0)[1]] for v in spviXValueToVoltageOverallAverageVectors]

In [77]:
prvXValueToStimulusPeaks = [[(0.25*i,v[i]) for i in findLocalExtremeIndices(v, minGap = (25 / 0.25), maxValue = -0.35)[0]] for v in prvXValueToStimulusOverallAverageVectors]

In [78]:
prvXValueToVoltagePeaks = [[(0.05*i,v[i]) for i in findLocalExtremeIndices(v, minValue = -61.0)[1]] for v in prvXValueToVoltageOverallAverageVectors]

In [81]:
[(t, v) for (t,v) in prvXValueToStimulusPeaks[0]]

[(40.75, -0.5551388899329252),
 (92.5, -0.6249382455006101),
 (174.0, -0.628512340472685),
 (253.0, -0.5225384610257596),
 (379.75, -0.5586334100889141),
 (473.25, -0.7393581322984274),
 (611.5, -0.4322355163032747),
 (709.75, -0.5089244621716491),
 (779.25, -0.7332698886961457),
 (895.0, -0.5706610274155689),
 (983.5, -0.5446123611157601),
 (1072.0, -0.36881286803469204),
 (1184.0, -0.4125106717058063),
 (1260.5, -0.44783612386924193),
 (1368.25, -0.45924058138016083),
 (1436.75, -0.6718973382531487),
 (1529.75, -0.6174459136097569),
 (1621.75, -0.4627384818528162),
 (1676.25, -0.6658669773790711),
 (1786.25, -0.6709153739061863),
 (1937.0, -0.39708748419846246)]

In [82]:
len(prvXValueToStimulusPeaks[0])

21

In [83]:
[(t, v) for (t,v) in prvXValueToVoltagePeaks[0]]

[(41.050000000000004, -58.03719643931015),
 (93.2, -58.42321960211492),
 (174.70000000000002, -57.99412865462342),
 (254.15, -58.420542116696),
 (380.5, -58.405198607886575),
 (473.45000000000005, -57.625247280738535),
 (613.45, -59.04868744372147),
 (710.6, -58.1035931281516),
 (779.75, -57.840783391310545),
 (895.7, -58.349432763773756),
 (984.45, -58.54479903328194),
 (1074.45, -60.4282248955983),
 (1186.55, -58.808777063614805),
 (1262.3000000000002, -58.77759564229942),
 (1369.75, -58.58154917496167),
 (1437.3000000000002, -58.158119423609705),
 (1530.3500000000001, -58.34347407165491),
 (1623.65, -58.968303219862875),
 (1677.1000000000001, -58.74737489829212),
 (1786.7, -58.22798969363959)]

In [84]:
len(prvXValueToVoltagePeaks[0])

20

In [85]:
[(t, v) for (t,v) in spviXValueToStimulusPeaks[0]]

[(40.75, -0.522772641282196),
 (92.5, -0.583903339950587),
 (174.0, -0.5866879845239593),
 (253.0, -0.4942322785359456),
 (379.75, -0.5259737533300506),
 (473.25, -0.6834037167007719),
 (611.5, -0.4165788041889842),
 (709.75, -0.48283131795209344),
 (779.5, -0.6779872338313156),
 (895.0, -0.5360973574785868),
 (983.5, -0.5142657857025763),
 (1072.0, -0.3621303529468506),
 (1184.0, -0.40022319261340844),
 (1260.5, -0.4291343100739725),
 (1368.25, -0.44025028658210313),
 (1436.75, -0.6240809973258711),
 (1529.75, -0.5778065429881803),
 (1621.75, -0.44282297706330515),
 (1676.25, -0.6188111513383249),
 (1786.25, -0.6244576857934243),
 (1937.0, -0.38614523056896044)]

In [86]:
len(spviXValueToStimulusPeaks[0])

21

In [36]:
[(t, v) for (t,v) in spviXValueToVoltagePeaks[0]]

[(41.150000000000006, -65.71101766818478),
 (95.5, -64.94527439208066),
 (176.60000000000002, -64.38346492140485),
 (255.10000000000002, -66.02118381620281),
 (381.0, -65.69102917182825),
 (474.3, -64.36943222526111),
 (614.85, -65.3319631206455),
 (712.25, -65.13248464069979),
 (780.7, -64.04167633679137),
 (896.7, -65.00173226265049),
 (986.1500000000001, -65.4588807477791),
 (1187.5, -66.09491512515638),
 (1263.6000000000001, -65.90217368810445),
 (1371.45, -66.2730733980849),
 (1438.0500000000002, -64.72291812523396),
 (1532.75, -65.16565699600436),
 (1624.75, -66.16889001546151),
 (1678.3500000000001, -64.73066690963853),
 (1789.15, -64.77327893032549),
 (1939.25, -66.36598380039717)]

In [35]:
len(spviXValueToVoltagePeaks[0])

20

In [90]:
#Assumes stims and volts are already sorted; generates a list of ((tstim, vstim), (tvolt, vvolt))
def divideToPairs(stims, volts, actionPotentialMaxTimeDifference = 5):
    for (tstim,vstim) in stims:
        (tvolt, vvolt) = min(volts, key=lambda (tvolt, vvolt): abs(tvolt-tstim))
        if(tvolt > tstim  and (tvolt - tstim) < actionPotentialMaxTimeDifference):
            yield ((tstim, vstim), (tvolt, vvolt))
        

In [95]:
prvXValueToPairs = map(lambda xvalue: list(divideToPairs(prvXValueToStimulusPeaks[xvalue], prvXValueToVoltagePeaks[xvalue])), range(0,20))
spviXValueToPairs = map(lambda xvalue: list(divideToPairs(spviXValueToStimulusPeaks[xvalue], spviXValueToVoltagePeaks[xvalue])), range(0,20))

In [97]:
spviXValueToPairs[0]

[((40.75, -0.522772641282196), (41.150000000000006, -65.71101766818478)),
 ((92.5, -0.583903339950587), (95.5, -64.94527439208066)),
 ((174.0, -0.5866879845239593), (176.60000000000002, -64.38346492140485)),
 ((253.0, -0.4942322785359456), (255.10000000000002, -66.02118381620281)),
 ((379.75, -0.5259737533300506), (381.0, -65.69102917182825)),
 ((473.25, -0.6834037167007719), (474.3, -64.36943222526111)),
 ((611.5, -0.4165788041889842), (614.85, -65.3319631206455)),
 ((709.75, -0.48283131795209344), (712.25, -65.13248464069979)),
 ((779.5, -0.6779872338313156), (780.7, -64.04167633679137)),
 ((895.0, -0.5360973574785868), (896.7, -65.00173226265049)),
 ((983.5, -0.5142657857025763), (986.1500000000001, -65.4588807477791)),
 ((1184.0, -0.40022319261340844), (1187.5, -66.09491512515638)),
 ((1260.5, -0.4291343100739725), (1263.6000000000001, -65.90217368810445)),
 ((1368.25, -0.44025028658210313), (1371.45, -66.2730733980849)),
 ((1436.75, -0.6240809973258711), (1438.0500000000002, -64.7

In [100]:
prvXValueToPairsRatios = map(lambda l: map(lambda i: ((l[i][0][1] / l[i-1][0][1]),(l[i][1][1] / l[i-1][1][1])), range(1, len(l))), prvXValueToPairs)
spviXValueToPairsRatios = map(lambda l: map(lambda i: ((l[i][0][1] / l[i-1][0][1]),(l[i][1][1] / l[i-1][1][1])), range(1, len(l))), spviXValueToPairs)
prvPairsRatios = flatten(prvXValueToPairsRatios)
spviPairsRatios = flatten(spviXValueToPairsRatios)

In [101]:
spviXValueToPairsRatios[0]

[(1.1169355353379944, 0.9883468054022413),
 (1.0047690163471372, 0.9913494942328811),
 (0.8424107729715437, 1.0254369487072057),
 (1.0642237995626918, 0.9949992619748582),
 (1.2993114435349655, 0.9798816221449319),
 (0.6095647331274071, 1.0149532295393877),
 (1.1590395696970057, 0.9969466939241157),
 (1.404190674099118, 0.9832524690263882),
 (0.7907189556491984, 1.0149911117380852),
 (0.9592768524756579, 1.0070328661901105),
 (0.778241920307092, 1.0097165483141697),
 (1.0722374864679332, 0.9970838689075106),
 (1.0259032574352176, 1.0056280345430761),
 (1.417559548162804, 0.9766095762069223),
 (0.9258518452957669, 1.0068405270280572),
 (0.7663862281191987, 1.01539511862082),
 (1.3974233122276778, 0.9782643610088227),
 (1.0091248104415822, 1.0006582972603457),
 (0.618368929318776, 1.0245889183992818)]

In [113]:
size = 10

plt.close(1)
#plt.rc('xtick', labelsize=2.65)
#plt.xticks(np.arange(0, (len(spviXValueToStimulusOverallAverageVectors[xvalue])+1)*0.25, 50.0))
plt.scatter(map(lambda (x,y): x, prvPairsRatios), map(lambda (x,y): y, prvPairsRatios), s = size)
plt.savefig(r".\data\ratios\PrV Ratios.pdf")
plt.close(1)

plt.close(1)
#plt.rc('xtick', labelsize=2.65)
#plt.xticks(np.arange(0, (len(spviXValueToStimulusOverallAverageVectors[xvalue])+1)*0.25, 50.0))
plt.scatter(map(lambda (x,y): x, spviPairsRatios), map(lambda (x,y): y, spviPairsRatios), s = size)
plt.savefig(r".\data\ratios\SpVi Ratios.pdf")
plt.close(1)